# IMPORTAÇÃO DAS BIBLIOTECAS

### A página a seguir apresenta um modelo analítico cujo objetivo é compreender um conjunto de variáveis acerca de clientes e encontrar semelhanças que agrupem os clientes em conjuntos, com características semelhantes entre si e diferentes entre os outros grupos.
### A presente página visa apresentar não apenas o modelo, como também a metodologia adotada e os conceitos envolvidos para escolha das técnicas utilizadas.

O código abaixo realiza a importação de todas as bibliotecas que foram utilizadas.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
from datetime import date
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix
from IPython.display import Image  
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import pydot 
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from itertools import cycle
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import DBScan

In [ ]:
data = pd.read_excel("Dataset - Code Challenge Data Scientist.xlsx")
print('Este dataset possui %s linhas e %s colunas' % (data.shape[0], data.shape[1]))

# ENTENDIMENTO DOS DADOS

A presente seção contém o entendimento das variáveis contidas neste modelo.
A tabela abaixo apresenta as variáveis que estão disponíveis para o desenvolvimento do modelo e uma pequena amostra dos dados.

In [ ]:
data.head()

A listagem abaixo apresenta a quantidade total de observações disponíveis para análise, a quantidade de registros com valor ausente e o tipo correspondente a cada variável.
Como pode ser observado, a base não contém nenhum valor ausente, o que significa que, provavelmente, não será necessário fazer tratamento de valores missings.

In [ ]:
data.info()

# VARIÁVEIS CATEGORICAS

A presente seção apresenta o detalhamento quanto ao conteúdo de cada variável categórica.
Quantidade de valores únicos, categoria com maior e com menor quantidade de observações etc.

A primeira variável a ser analisada é a variável DATA_NASCIMENTO.
Na análise é possível observar que dentre os 4972 valores preenchidos, 3982 são únicos, ou seja, apenas 990 se repetem.
Também é possível observar que apenas 5 pessoas nasceram no mesmo dia, dia 05 de janeiro de 1977.
A pessoa mais velha dentre os clientes tem 93 anos e a penas mais jovem tem 2 anos de idade.

#### A partir desta variável, surgiu a necessidade de criação da variável ANO, que provavelmente tem categorias com maior número de repetições dentre as observações.

In [ ]:
data["DATA_NASCIMENTO"].describe()
# NECESSIDADE DE CRIAÇÃO DA VARIÁVEL ANO

A próxima variável que será analisada é a variável PERFIL. Foi observado que esta variável contém 4 categorias e mais recorrente é o perfil A, para este perfil ocorreram 3489 observações, ou seja, 70% dos clientes pertencem a um mesmo perfil.

In [ ]:
data["PERFIL"].describe()

A próxima variável a ser analisada é a variável PROFISSÃO. Esta variável contém 79 níveis e o nível mais recorrente é a profissão ANALISTA DE SISTEMAS, com 564 observações.

#### Será necessário redistribuir as categorias desta variável, devido a alta quantidade de níveis e a baixa quantidade de observações em algumas categorias.

In [ ]:
data["PROFISSAO"].describe()
# NECESSIDADE DE REDISTRIBUIÇÃO, MUITAS CATEGORIAS E ALGUMAS COM POUCAS OBSERVAÇÕES

In [ ]:
data["PROFISSAO"].value_counts()

A próxima variável a ser analisada é a variável ESTADO_CIVIL. Esta variável contém 8 níveis e a categoria mais recorrente é o status SOLTEIRO(A), que corresponde a 2584 observações, quase metade da base.

#### Também será necessário redistribuir as categorias desta variável, devido a grande diferença de observações entre as categorias existentes.

In [ ]:
data["ESTADO_CIVIL"].describe()
# NECESSIDADE REDISTRIBUIÇÃO DAS CATEGORIAS, CATEGORIAS COM POUCAS OBSERVAÇÕES

In [ ]:
data["ESTADO_CIVIL"].value_counts()

A última variável categórica a ser analisada é a variável GENERO. Esta variável contém apenas 2 níveis e a maioria dos registros estão contidos em uma das categorias.

In [ ]:
data["GENERO"].describe()

# VARIÁVEIS CONTÍNUAS

Esta seção visa analisar as informações estatísticas das variáveis contínuas.

A primeira variável a ser análisa é a GEO_REFERENCIA.
As informações estatísticas desta variável estão dentro da normalidade, sem valores muito afastados da média, transição entre os quartis gradativa, aparentemente sem outliers.

In [ ]:
data["GEO_REFERENCIA"].describe()

A próximas 4 variáveis a serem analisadas são as variáveis VALOR_01, VALOR_02, VALOR_03 e VALOR_04.
Ambas são variáveis com observações muito acima ou abaixo da média, transição entre os quartis muito diferentes, o que indica a presença de outliers, principalmente nas variáveis 2, 3 e 4.
#### Os outliers destas 4 variáveis serão tratados adiante.

In [ ]:
data["VALOR_01"].describe()
# PRESENÇA DE OUTLIERS

In [ ]:
data["VALOR_02"].describe()
# PRESENÇA DE OUTLIERS

In [ ]:
data["VALOR_03"].describe()
# PRESENÇA DE OUTLIERS

In [ ]:
data["VALOR_04"].describe()
# PRESENÇA DE OUTLIERS

# EXCLUSÃO E CRIAÇÃO DE VARIÁVEIS

Esta seção tem a finalidade de excluir variáveis que não serão utilizadas no modelo e a criação de novas variáveis a partir das variáveis disponíveis.

A variável ANO será criada a partir da variável DATA_NASCIMENTO, pois poucas pessoas têm a mesma data de nascimento, mas muitas pessoas várias pessoas têm a mesma idade, portanto será mais fácil encontrar padrões com esta última variável.
As variáveis ID e DATA_NASCIMENTO serão excluídas, devido a alta granularidade de ambas.
A variável DATA_NASCIMENTO também não pode ser utilizada devido a alta colinearidade com a variável ANO.

In [ ]:
data_aux = data.copy()

In [ ]:
data_aux["ANO"] = list(map(lambda var: var.date().year,data_aux["DATA_NASCIMENTO"]))

In [ ]:
data_aux = data_aux.drop(["ID","DATA_NASCIMENTO"],axis=1)
data_aux.head()

In [ ]:
data1 = data_aux.copy()

# FAZER ANÁLISE DE OUTLIERS

Esta seção tem a finalidade de compreender o quanto os outliers encontrados nas análises anteriores podem interferir na base de dados e qual a melhor estratégias para eliminá-los.

Primeiramente foi observada a grande variação de valores de cada uma das 4 variáveis.
Nos gráficos abaixo, é possível observar que a variável VALOR_01 tem um padrão de valores e alguns valores muito fora da média, enquanto as variáveis VALOR_02, VALOR_03 e VALOR_04 têm vários valores fora da média.

Um vez identificada e confirmada a existência de outliers e seu impacto na modelagem dos dados, foi considerada a possibilidade de exclusão dos registros com valores muito discrepantes, entretanto depois da realização de alguns testes percebeu-se que a remoção dos valores discrepantes para as 4 variáveis resumiria a base em uma quantidade muito pequena de observações, por isso optou-se em aplicar o valor da média nos valores discrepantes.

Para encontrar os valores discrepantes foi verificada a distância entre a cada observação e a média, quando este valor era maior que duas vezes o desvio padrão, o valor da variável foi substituído pela média da variável.

In [ ]:
data_aux = data1.copy()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20,20))
axes[0].plot(data_aux["VALOR_01"])
axes[1].plot(data_aux["VALOR_02"])
axes[2].plot(data_aux["VALOR_03"])
axes[3].plot(data_aux["VALOR_04"])

In [ ]:
def outliers(var):
    if np.abs(var-variavel.mean())<=0.5*variavel.std():
        return var
    else:
        return variavel.mean()

In [ ]:
# VALOR_01
variavel = data_aux["VALOR_01"]
data_aux["VALOR_01"] = list(map(outliers,variavel))
data_aux["VALOR_01"].describe()

In [ ]:
# VALOR_02
variavel = data_aux["VALOR_02"]
variavel = list(map(outliers,variavel))
data_aux["VALOR_02"] = variavel
data_aux["VALOR_02"].describe()

In [ ]:
# VALOR_03
variavel = data_aux["VALOR_03"]
variavel = list(map(outliers,variavel))
data_aux["VALOR_03"] = variavel
data_aux["VALOR_03"].describe()

In [ ]:
# VALOR_04
variavel = data_aux["VALOR_04"]
variavel = list(map(outliers,variavel))
data_aux["VALOR_04"] = variavel
data_aux["VALOR_04"].describe()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20,5))
axes[0].plot(data_aux["VALOR_01"])
axes[1].plot(data_aux["VALOR_02"])
axes[2].plot(data_aux["VALOR_03"])
axes[3].plot(data_aux["VALOR_04"])

In [ ]:
data2 = data_aux.copy()

# RECATEGORIZAR VARIÁVEIS

Esta seção tem o objetivo de redistribuir as categorias de variáveis com muitos níveis ou categorias com pocas observações.

In [ ]:
data_aux = data2.copy()

In [ ]:
def estudo_recat(coluna,w,h):
    lista = data_aux[coluna].unique()
    fig, axes = plt.subplots(lista.size, 4, figsize=(w,h))
    i=0
    for j in lista:
        axes[i][0].plot(data_aux[data_aux[coluna]==j]["VALOR_01"])
        axes[i][0].set_xlabel(j)
        axes[i][0].set_ylabel("VALOR_01")
        axes[i][1].plot(data_aux[data_aux[coluna]==j]["VALOR_02"])
        axes[i][1].set_xlabel(j)
        axes[i][1].set_ylabel("VALOR_02")
        axes[i][2].plot(data_aux[data_aux[coluna]==j]["VALOR_03"])
        axes[i][2].set_xlabel(j)
        axes[i][2].set_ylabel("VALOR_03")
        axes[i][3].plot(data_aux[data_aux[coluna]==j]["VALOR_04"])
        axes[i][3].set_xlabel(j)
        axes[i][3].set_ylabel("VALOR_04")
        i=i+1

In [ ]:
def aplica_cat_outras(var):
    if var in lista:
        return "OUTROS"
    else:
        return var

In [ ]:
def aplica_faixa_etaria(var):
    idade = datetime.now().year-var
    if idade < 18:
        return "infantil"
    elif (idade >= 18 and idade <30) or (idade >= 40):
        return "jovem/idoso"
    elif idade >= 30 and idade < 40:
        return "adulto"
    else:
        return var

A primeira variável a sofre recategorização foi a PROFISSAO, devido a alta quantidade de níveis existente.
Foi observado que muitas categorias continham menos de 100 observações e para esta quantidade tão pequena de observações não foi possível realizar cruzamento com outros dados, pois era informações muito fora do contexto.
Portanto, todas as observações cujo PROFISSAO tinha uma frequência menor que 100 na base foram recategorizadas para a categoria OUTROS.

Com esta modificação, a variável PROFISSAO passou a ter 15 níveis, e o nível mais recorrente passou a ser o nível OUTROS.

In [ ]:
freq_profissoes = data_aux['PROFISSAO'].value_counts()
lista = freq_profissoes[freq_profissoes<100]
data_aux['PROFISSAO'] = list(map(aplica_cat_outras,data_aux['PROFISSAO']))
data_aux["PROFISSAO"].value_counts()

A próxima variável a sofrer alteração foi a variável ESTADO_CIVIL.
Esta variável foi apresentada no gráfico separada por cada um dos níveis e em contraponto com as variáveis VALOR_01, VALOR_02, VALOR_03 e VALOR_04, em gráficos diferentes.
#### Nesta análise foi possível observar que os níveis SOLTEIRO(A) e CASADO(A) COM BRASILEIRO(A) NATO(A) tiham cada um padrão próprio, enquanto os demais níveis tinham um outro padrão semelhante entre si.
Por isso, estes valores foram agrupados, utilizando como recurso a frequência de observações por nível.
Pode-se observar uma sútil semelhança entre os gráficos destas 3 categorias resultantes.

In [ ]:
freq_estadocivil = data_aux['ESTADO_CIVIL'].value_counts()
lista = freq_estadocivil[freq_estadocivil<1000]
data_aux['ESTADO_CIVIL'] = list(map(aplica_cat_outras,data_aux['ESTADO_CIVIL']))
data_aux["ESTADO_CIVIL"].value_counts()

In [ ]:
estudo_recat("ESTADO_CIVIL",30,15)

A proxima variável a ser estudad foi a FAIXA_ETARIA.
Esta variável primeiramente foi criada com base na variável ANO. A partir da variável intermediária IDADE, as observações foram agrupadas em INFANTIL (abaixo de 18 anos), JOVEM (entre 18 e 30 anos), ADULTO1 (entre 30 e 35 anos), ADULTO2 (entre 35 e 40 anos), ADULTO3 (acima de 40 anos). Este agrupamento foi realizado baseado nas categorias sociais (criança, jovem, adulto) e no equilíbrio da quantidade de observações em cada nível.

Uma vez criada as categorias, os gráficos de contraponto com as variáveis númericas VALOR_01, VALOR_02, VALOR_03 e VALOR_04 mostraram leve semelhança entre o padrão dos níveis JOVEM e ADULTO3 e ADULTO1 e ADULTO2, por tanto estes níveis também foram agrupados.

Desde a primeira categorização desta variável, ficou evidente a baixa quantidade de observações no nível INFANTIL, apenas 13 registros, portanto estes registros foram excluídos da base de dados, para não criar ruído nos outros perfis de clientes.

In [ ]:
data_aux["FAIXA_ETARIA"]=list(map(aplica_faixa_etaria,data_aux["ANO"]))
# REMOÇÃO DE CRIANÇAS DA BASE
data_aux = data_aux[data_aux["FAIXA_ETARIA"] != "infantil"]
data_aux["FAIXA_ETARIA"].value_counts()

In [ ]:
estudo_recat("FAIXA_ETARIA",20,10)

In [ ]:
data3 = data_aux.copy()

# CRIAÇÃO DE VARIÁVEIS DUMMIES

In [ ]:
data_aux = data3.copy()

Para visualização de alguns gráficos e utilização de modelos que só aceitam valores numéricos, foram criadas variáveis dummies a partir das variáveis categoricas.

In [ ]:
profissao = pd.get_dummies(data_aux['PROFISSAO'],drop_first=True)
genero = pd.get_dummies(data_aux['GENERO'],drop_first=True)
estado_civil = pd.get_dummies(data_aux['ESTADO_CIVIL'],drop_first=True)
perfil = pd.get_dummies(data_aux['PERFIL'],drop_first=True)
faixaetaria = pd.get_dummies(data_aux['FAIXA_ETARIA'],drop_first=True)

base_com_dummies = pd.concat([data_aux.drop(["PROFISSAO","GENERO","ESTADO_CIVIL","PERFIL","FAIXA_ETARIA"],axis=1),profissao,genero,estado_civil,perfil,faixaetaria],axis=1)

In [ ]:
base_dummies_profissao = data_aux.copy()
base_dummies_profissao.drop(["GENERO","ESTADO_CIVIL","PERFIL","FAIXA_ETARIA"], axis=1,inplace=True)
base_dummies_profissao = pd.concat([base_dummies_profissao,genero,estado_civil,perfil,faixaetaria],axis=1)

In [ ]:
base_dummies_genero = data_aux.copy()
base_dummies_genero.drop(["ESTADO_CIVIL","PERFIL","FAIXA_ETARIA","PROFISSAO"], axis=1,inplace=True)
base_dummies_genero = pd.concat([base_dummies_genero,estado_civil,perfil,faixaetaria,profissao],axis=1)

In [ ]:
base_dummies_estadocivil = data_aux.copy()
base_dummies_estadocivil.drop(["GENERO","PERFIL","FAIXA_ETARIA","PROFISSAO"], axis=1,inplace=True)
base_dummies_estadocivil = pd.concat([base_dummies_estadocivil,genero,perfil,faixaetaria,profissao],axis=1)

In [ ]:
base_dummies_faixaetaria = data_aux.copy()
base_dummies_faixaetaria.drop(["GENERO","ESTADO_CIVIL","PERFIL","PROFISSAO"], axis=1,inplace=True)
base_dummies_faixaetaria = pd.concat([base_dummies_faixaetaria,genero,estado_civil,perfil,profissao],axis=1)

In [ ]:
base_dummies_perfil = data_aux.copy()
base_dummies_perfil.drop(["GENERO","ESTADO_CIVIL","FAIXA_ETARIA","PROFISSAO"], axis=1,inplace=True)
base_dummies_perfil = pd.concat([base_dummies_perfil,genero,estado_civil,faixaetaria,profissao],axis=1)

In [ ]:
data4 = data_aux.copy()

# VISUALIZAÇÃO DOS DADOS

Esta seção tem o objetivo de oferecer insights quanto ao cruzamento das variáveis do modelo e visualizações isoladas de cada variável. Estas visualizações foram realizadas em vários momentos no decorrer do desenvolvimento do modelo, porém, a título de organização do código, todas as visualizações foram organizadas nesta seção.

In [ ]:
data_aux = data4.copy()
data_aux.head(1)

#### No grid abaixo é possível observar a correlação de todas as variáveis contínuas entre si.
#### Algumas observações são dignas de destaque, são elas:
##### 1. A variável GEO_REFERENCIA não tem correlação com ennhuma outra variável numérica
##### 2. As variáveis VALOR_01 e VALOR_02 tâm alta correlação
##### 3. As variáveis VALOR_01 e ANO têm correlação inversa, o que significa que quanto mais novo o cliente mais alto o valor da variável VALOR_01, fenomeno que ocorre em menor grau nas variáveis VALOR_02, VALOR_03 e VALOR_04
##### 4. As variáveis VALOR_03 e VALOR_04 também têm correlação com a variável VALOR_02, em menor grau

#### A partir das conclusões obtidas pode ser possível inferir diversas regras de negócio, a depender do significado das variáveis e do contexto em que será aplicado. Por exemplo, caso as variáveis VALOR_N correspondam a um valor em dinheiro utilizado em viagens, é possível inferir que clientes com maior idade têm um gasto maior que cliente mais novos. 

In [ ]:
sns.heatmap(data_aux.corr(),annot=True)

#### Os proximos 4 gráficos apresentam os valores VALOR_N em contraponto a FAIXA_ETARIA dos clientes.

#### É possível observar uma concentração ligeiramente maior na coluna ADULTO em relação a coluna JOVEM/IDOSO, para as variáveis VALOR_01 e VALOR_02.

In [ ]:
sns.stripplot(x="FAIXA_ETARIA", y="VALOR_01",data=data_aux,jitter=True)

In [ ]:
sns.stripplot(x="FAIXA_ETARIA", y="VALOR_02", data=data_aux,jitter=True)

In [ ]:
sns.stripplot(x="FAIXA_ETARIA", y="VALOR_03", data=data_aux,jitter=True)

In [ ]:
sns.stripplot(x="FAIXA_ETARIA", y="VALOR_04", data=data_aux,jitter=True)

#### Os proximos 4 gráficos apresentam os valores VALOR_N em contraponto a GENERO dos clientes.

#### É possível observar uma concentração maior na coluna M em relação a coluna F, entretanto, isto ocorre devido maior quantidade de observações com GENERO com valor M.

In [ ]:
sns.stripplot(x="GENERO", y="VALOR_01", hue='PERFIL',data=data_aux,jitter=True)

In [ ]:
sns.stripplot(x="GENERO", y="VALOR_02", hue='PERFIL',data=data_aux,jitter=True)

In [ ]:
sns.stripplot(x="GENERO", y="VALOR_03", hue='PERFIL',data=data_aux,jitter=True)

In [ ]:
sns.stripplot(x="GENERO", y="VALOR_04", hue='PERFIL',data=data_aux,jitter=True)

#### Os proximos 4 gráficos apresentam os valores VALOR_N em contraponto a ESTADO_CIVIL dos clientes.
#### É possível observar uma concentração ligeiramente maior no topo da coluna CASADO(A) COM BRASILEIRO(A) NATIVO(A) em relação às outras colunas, para as variáveis VALOR_01 e VALOR_02. Este fato é interessante, porque mesmo a categoria SOLTEIRO(A) contendo mais registros, as observações com maiores valores nas variáveis VALOR_N estão na categoria CASADO(A) COM BRASILEIRO(A) NATIVO(A), o que mais uma vez pode evidenciar um padrão de consumo dos clientes, mostrando que pessoas casadas consomem um valor maior que pessoas solteiras ou em outros tipos de relacionamentos.

In [ ]:
sns.stripplot(x="ESTADO_CIVIL", y="VALOR_01", hue='PERFIL',data=data_aux,jitter=True)

In [ ]:
sns.stripplot(x="ESTADO_CIVIL", y="VALOR_02", hue='PERFIL',data=data_aux,jitter=True)

In [ ]:
sns.stripplot(x="ESTADO_CIVIL", y="VALOR_03", hue='PERFIL',data=data_aux,jitter=True)

In [ ]:
sns.stripplot(x="ESTADO_CIVIL", y="VALOR_04", hue='PERFIL',data=data_aux,jitter=True)

#### Os proximos 4 gráficos apresentam os valores VALOR_N em contraponto a PERFIL dos clientes.
#### É possível observar uma concentração maior nas colunas A e B em relação às outras colunas. Nesta variável ocorre o mesmo fato da variável ESTADO_CIVIL, pois foi possível observar nas análises anteriores que o PERFIL A contém a maioria das observações da base, mas para altos valores de VALOR_N temos alta concentração no PERFIL A e PERFIL B.

In [ ]:
sns.stripplot(x="PERFIL", y="VALOR_01", hue='PERFIL',data=data_aux,jitter=True)

In [ ]:
sns.stripplot(x="PERFIL", y="VALOR_02", hue='PERFIL',data=data_aux,jitter=True)

In [ ]:
sns.stripplot(x="PERFIL", y="VALOR_03", hue='PERFIL',data=data_aux,jitter=True)

In [ ]:
sns.stripplot(x="PERFIL", y="VALOR_04", hue='PERFIL',data=data_aux,jitter=True)

In [ ]:
data_aux.head(1)

#### A análise abaio apresenta a presença de valores altos das variáveis contínuas nas profissões dos clientes.
#### É  possível observar que para as profissões BANCARIO E ECONOMIARIO, ENGENHEIRO, MÉDICO e SERVIDOR PÚBLICO FEDERAL a média de VALOR_2 é bem alta. em comparação às outras profissões.
#### Também é possível observar este padrão segue nas outras variáveis VALOR_N, com menor intensidade.
#### O ano de nascimento (IDADE) não tem nenhuma influência em relação a profissão dos clientes.

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(base_dummies_profissao.drop(["M","OUTROS","SOLTEIRO(A)","B","C","D","jovem/idoso"],axis=1).groupby("PROFISSAO").mean())

#### Em relação ao GENERO é possível observar que a presença de homens nas profissões de ESTUDANTE, SERVIDOR PUBLICO FEDERAL, AUTÔNOMO é mais alta que em outras profissões.

In [ ]:
plt.figure(figsize=(10,2))
sns.heatmap(base_dummies_genero.drop(["GEO_REFERENCIA","VALOR_01","VALOR_02","VALOR_03","VALOR_04","ANO","SOLTEIRO(A)","jovem/idoso","B","OUTROS","ANALISTA DE SISTEMAS","ENGENHEIRO"],axis=1).groupby("GENERO").sum())

#### Em relação a faixa etária, não há diferença relevante na profissão de ENGENHEIRO
#### Nas profissões de ANALISTA DE SISTEMAS e SERVIDOR PUBLICO FEDERAL existe uma predominância de ADULTOS (entre 30 e 40 anos)
#### Nas profissões de AUTÔNOMO, AUXILIAR DE ESCRITÓRIOS, EMPRESÁRIO e ESTUDANTE, existe uma predominância de JOVENS ou ADULTOS acima de 40 anos.

In [ ]:
plt.figure(figsize=(10,3))
sns.heatmap(base_dummies_faixaetaria.drop(["GEO_REFERENCIA","VALOR_01","VALOR_02","VALOR_03","VALOR_04","ANO","M","OUTROS","SOLTEIRO(A)","B"],axis=1).groupby("FAIXA_ETARIA").sum())

#### Quanto ao ESTADO_CIVIL, é possível observar uma predominância de pessoas solteiras nas profissões ANALISTAS DE SISTEMAS e ESTUDANTES.

In [ ]:
plt.figure(figsize=(10,3))
sns.heatmap(base_dummies_estadocivil.drop(["GEO_REFERENCIA","VALOR_01","VALOR_02","VALOR_03","VALOR_04","ANO","M","jovem/idoso","B","OUTROS"],axis=1).groupby("ESTADO_CIVIL").sum())

## DISTRIBUIÇÕES DAS VARIÁVEIS CONTÍNUAS

#### A próxima seção apresenta a distribuição das variáveis contínuas do modelo. Esta informação é útil na decisão da técnica de modelagem que será escolhida e da necessidade de novas categorização nos dados.

In [ ]:
sns.distplot(data_aux["VALOR_01"])

In [ ]:
sns.distplot(data_aux["VALOR_02"])

In [ ]:
sns.distplot(data_aux["VALOR_03"])

In [ ]:
sns.distplot(data_aux["VALOR_04"])

In [ ]:
sns.distplot(data_aux["GEO_REFERENCIA"])

# APLICAR TÉCNICAS DE MODELAGEM

Esta seção será utilizada para aplicar diferentes técnicas de modelagem nos dados análisados e tratados.
Os resultados serão comparados a fim de encontrar o modelo mais acertivo no agrupamento dos dados analisados, de modo que os grupos encontrados reúnam observações o mais semelhantes dentro das observações dentro do seu próprio grupo e o mais diferentes quando comparados a observações de outros grupos.

A base que será utilizada será a base de variáveis dummies, devido a facilidade em manipulação de valores numéricos para a maioria das técnicas de modelagem.

Foi necessário excluir algumas variáveis, devido a baixa correlação com outras variáveis do modelo, no caso de GEO_REFERENCIA, e a alta colinearidade com outras variáveis, no caso do ANO.

In [ ]:
data_aux = base_com_dummies.copy()
data_aux=data_aux.drop(["GEO_REFERENCIA","ANO"],axis=1)
data_aux.head()

#### A primeira técnica que será utilizada será o Kmeans, por ser um algoritmo recomendado para uso geral e por ser interessante em dados com poucos clusters.

In [ ]:
#lista = range(1,1000)
#for i in lista:
i=2
kmeans = KMeans(n_clusters=i)
kmeans.fit(data_aux)
#    erro = np.abs(data_aux[[]]-kmeans.cluster_centers_[i])

#### O Gráfico abaixo apresenta os centróides resultantes do agrupamento do modelo

In [ ]:
plt.scatter(data_aux["VALOR_01"], data_aux["VALOR_02"], s = 100, c = kmeans.labels_)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 300, c = 'red',label = 'Centroids')
plt.legend()

plt.show()

#### O Gráfico abaixo apresenta um overview geral do agrupamento dos clientes utilizando a técnica Kmeans.
#### Como é possível observar, apenas a variável VALOR_02 foi considerada, pois o agrupamento não distíngue os grupos em nenhuma outra variável.

In [ ]:
base_sem_dummies = data4.copy()
base_sem_dummies["LABEL"] = kmeans.labels_
sns.pairplot(base_sem_dummies,hue='LABEL',palette='rainbow')

#### Uma possível razão para o modelo não ter resultado em um melhor agrupamento dos dados, pode ser a necessidade de grupos de mesmo tamanho do Kmeans.

#### A próxima técnica que será aplicada é a AffinityPropagation. Este algoritmo é recomendado em casos em que os clusters têm tamanhos diferentes e quando tem-se uma grande quantidade de grupos.

In [ ]:
X = data_aux
af = AffinityPropagation(preference=-50).fit(X)
cluster_centers_indices = af.cluster_centers_indices_
labels = af.labels_

n_clusters_ = len(cluster_centers_indices)

print('Estimated number of clusters: %d' % n_clusters_)
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels, metric='sqeuclidean'))
plt.close('all')
plt.figure(1)
plt.clf()

colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    class_members = labels == k
    cluster_center = X[cluster_centers_indices[k]]
    plt.plot(X[class_members, 0], X[class_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)
    for x in X[class_members]:
        plt.plot([cluster_center[0], x[0]], [cluster_center[1], x[1]], col)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

#### O útimo algoritmo que será testado é o DBScan. Este algoritmo utiliza geometria não plana para agrupar as observações, permitindo deste modo a utlização de várias variáveis na identificação dos grupos.

In [ ]:
X = data_aux
X = StandardScaler().fit_transform(X)

db = DBSCAN(eps=0.3, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters_)
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Cluster estimados: %d' % n_clusters_)
plt.show()

#### O algoritmo abaixo é o MeanShift...

In [ ]:
X = data_aux
bandwidth = estimate_bandwidth(X, quantile=0.2, n_samples=500)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)

plt.figure(1)
plt.clf()

colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    my_members = labels == k
    cluster_center = cluster_centers[k]
    plt.plot(X[my_members, 0], X[my_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)
plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

# AVALIAÇÃO DO MODELO

In [ ]:
X = data_aux
y = kmeans.labels_

dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
predictions = dtree.predict(X_test)
print(classification_report(y_test,predictions))

In [ ]:
print(confusion_matrix(y_test,predictions))

In [ ]:
from IPython.display import Image  
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import pydot 

features = list(.columns[1:])
dot_data = StringIO()  
export_graphviz(dtree, out_file=dot_data,feature_names=features,filled=True,rounded=True)

graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph[0].create_png())

# CONCLUSÕES

Neste modelo foi possível realizar uma análise aprofundada em uma base de dados de clientes.
Inicialmente foi realizado o entendimento dos dados e das váriáveis disponíveis na base de dados.
Em seguida foram realizados diversos tratamentos nos dados, a fim de obter um melhor aproveitamento das variáveis e perder a menor quantidade de informações possível. Dentre os tratamentos realizados estão: tratamento de outliers, exclusão de categorias muito pequenas, redistribuição de categorias.
Em seguida foram realizadas 
http://scikit-learn.org/stable/modules/clustering.html#k-means